In [ ]:
%matplotlib inline
import gmaps
import json
import requests
import time


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from  citipy import citipy
import matplotlib.pyplot as plt
from scipy import stats

import requests
from census import Census
import gmaps
import seaborn as sns
import scipy.stats as st
import warnings
import matplotlib.image as mpimg
import matplotlib

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#%matplotlib notebook
from api_keys import g_key
from api_keys import api_key
from api_keys import weather_api_key

lat_range = (-90, 90)
lng_range = (-180, 180)
lng_range = (-180, 180)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
abnb_data = pd.read_csv("Resources/airbnb_data.csv")

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
abnb2 = abnb_data.drop(['neighbourhood_group', 'id', 'host_id', 'host_name', 'calculated_host_listings_count', 'last_review'], axis=1)
abnb2_df = abnb2.drop(abnb2[abnb2['room_type'] == 'Private room'].index)

abnb2_df['city'] = abnb2_df['city'].replace({'Broward County':'Fort Lauderdale','Clark County':'Las Vegas','New York City':'New York','San Clara Country':'San Jose','San Mateo County':'San Mateo','Santa Cruz County':'Santa Cruz','Twin Cities MSA':'Twin Cities','Washington D.C.':'Washington'})
abnb_df = abnb2_df.drop(abnb2_df[(abnb2_df['city'] == 'Hawaii') | (abnb2_df['city'] == 'Rhode Island') | (abnb2_df['city'] == 'Twin Cities')].index)

abnb_df = abnb_df[['city', 'name', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']]
abnb_df.rename(columns={'city': 'City', 'name': 'Property Name', 'neighbourhood': 'Neighborhood', 'latitude': 'Latitude', 'longitude': 'Longitude', 'room_type': 'Room Type', 'price': 'Price', 'minimum_nights': 'Minimum Nights', 'number_of_reviews': 'Number of Reviews', 'reviews_per_month': 'Reviews per Month', 'availability_365': 'Availability (days per year)'}, inplace=True)

abnb_df

In [ ]:
Cities = []
for city in abnb_df['City']:
    if city not in Cities:
        Cities.append(city)

In [ ]:
cities = [city.lower() for city in Cities]

In [ ]:
states = ['NC', 'TX', 'MA', 'FL', 'MA', 'IL', 'NV', 'OH', 'CO', 'NJ', 'CA', 'TN', 'LA', 'NY', 'CA', 'CA', 'OR', 'OR', 'CA', 'CA', 'CA', 'CA', 'CA', 'WA', 'DC']

In [ ]:
loop_df = pd.DataFrame({
    'city': cities,
    'City': Cities,
    'State': states
})

loop_df['Population'] = ''

In [ ]:
for index, row in loop_df.iterrows():
    city = row["city"]
    City = row["City"]
    state = row["State"]
    url = f'http://public.opendatasoft.com/api/records/1.0/search/?dataset=cities-and-towns-of-the-united-states%40public&q={city}&facet=name&facet=state&refine.name={City}&refine.state={state}'
    response = requests.get(url).json()
    print(f"Searching population record for {city}")
    
    try:
        loop_df.loc[index, "Population"] = response['records'][0]['fields']['pop_2010']
    
        print(f"{city}'s population: {response['records'][0]['fields']['pop_2010']}")
    
        print('________________________________________________')
    
    except (KeyError, IndexError):
        print(f"Population for {city} not found... Continue scanning.")
        
        print('________________________________________________')

In [ ]:
graphs_draft_df = abnb_df.groupby('City').mean().reset_index()
graphs_draft_df['Population'] = loop_df['Population']
graphs_draft_df['State'] = loop_df['State']
graphs_df = graphs_draft_df[['City', 'State', 'Population', 'Latitude', 'Longitude', 'Price', 'Minimum Nights', 'Number of Reviews', 'Reviews per Month', 'Availability (days per year)']]
graphs_df['Population'] = pd.to_numeric(graphs_df['Population'], errors='coerce')

graphs_df

In [ ]:
# 'sort by population
sort_Population_graphs_df = graphs_df.sort_values(by='Population', ascending=False)
sort_Population_graphs_df.head()

In [ ]:
# Average prices for each type of listing

avgPrice_DF=abnb2_df.groupby('room_type').price.mean()
avgPrice_DF=abnb2_df.reset_index()
avgPrice_DF=abnb2_df.rename(columns={'price':'average_Price'})
avgPrice_DF

In [ ]:

#Number of Review sorted by population as it related to price
sns.jointplot(data=sort_Population_graphs_df, x='Number of Reviews', y='Price')

In [ ]:
sns.jointplot(data=abnb_df, x='Price', y='City')

In [ ]:
sns.jointplot(data=abnb_df, x='Neighborhood', y='Price')

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(13,7))
plt.title("Neighbourhood Group")
x = plt.pie(graphs_df.City.value_counts(), labels=graphs_df.City.value_counts().index,autopct= '%1.1f%%', startangle=180)

plt.show()